<a href="https://colab.research.google.com/github/First-time258/Fintech_Projects/blob/main/Fintech_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing and Importing libraries

---



---



In [1]:
%pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29413 sha256=8185fa1f456f561ca968b8bf5fa5e8632b0a5eaaa88381e9959b97b2dae02063
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [2]:
%pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=78a7b94356d3aec138972a432086d16c075ecc3a021be07f605e5c1cc8c58852
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [3]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
import ta as ta
from pandas_datareader import data
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt


/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


# Downloading the Data and Cleaning

---



In [5]:
df=yf.download('TATAMOTORS.NS',start='2018-01-01')
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-01,430.950012,436.399994,422.250000,424.450012,423.131256,6807536
2018-01-02,428.850006,440.850006,422.000000,439.299988,437.935089,15331261
2018-01-03,440.399994,441.399994,431.950012,433.899994,432.551849,9794953
2018-01-04,430.000000,433.299988,425.750000,429.950012,428.614166,8395377
2018-01-05,431.250000,436.350006,429.799988,431.600006,430.259033,7021611
...,...,...,...,...,...,...
2024-01-19,823.849976,826.000000,819.500000,823.549988,823.549988,5706888
2024-01-22,823.549988,823.549988,823.549988,823.549988,823.549988,0
2024-01-23,824.900024,827.599976,796.299988,800.450012,800.450012,9436209


# **Implementing the Strategy**

---



In [6]:
class SMAcross(Strategy):
  n1=50
  n2=100
  def init(self):
    close=self.data.Close
    self.SMA_1=self.I(ta.trend.sma_indicator,pd.Series(close),self.n1)
    self.SMA_2=self.I(ta.trend.sma_indicator,pd.Series(close),self.n2)
  def next(self):
    if crossover(self.SMA_1,self.SMA_2):
      self.buy_price = self.data.Close[-1]
      self.buy()
    elif crossover(self.SMA_2,self.SMA_1):

     self.position.close()
bt=Backtest(df,SMAcross,cash=1000,commission=0.02)
stat=bt.run()
stat

Start                     2018-01-01 00:00:00
End                       2024-01-25 00:00:00
Duration                   2215 days 00:00:00
Exposure Time [%]                        54.4
Equity Final [$]                  1092.810016
Equity Peak [$]                   1148.957052
Return [%]                           9.281002
Buy & Hold Return [%]               91.271045
Return (Ann.) [%]                    1.502211
Volatility (Ann.) [%]               26.376298
Sharpe Ratio                         0.056953
Sortino Ratio                        0.086136
Calmar Ratio                         0.024063
Max. Drawdown [%]                  -62.429398
Avg. Drawdown [%]                  -18.413649
Max. Drawdown Duration      955 days 00:00:00
Avg. Drawdown Duration      223 days 00:00:00
# Trades                                    7
Win Rate [%]                        28.571429
Best Trade [%]                     168.652929
Worst Trade [%]                    -49.732514
Avg. Trade [%]                    

In [7]:
stat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,5,296,305,185.130000,169.000000,-80.650000,-0.087128,2019-03-14,2019-03-28,14 days
1,4,308,368,209.100000,163.750000,-181.400000,-0.216882,2019-04-03,2019-07-03,91 days
2,4,461,540,170.288997,85.599998,-338.755994,-0.497325,2019-11-21,2020-03-16,116 days
3,3,622,900,106.028997,284.850006,536.463028,1.686529,2020-07-16,2021-08-26,406 days
4,1,936,1032,522.699012,390.500000,-132.199012,-0.252916,2021-10-19,2022-03-08,140 days
5,1,1133,1190,488.477994,408.549988,-79.928006,-0.163627,2022-08-02,2022-10-27,86 days
6,1,1269,1499,444.720000,814.000000,369.280000,0.830365,2023-02-17,2024-01-25,342 days


# Optimizing it

---



In [8]:
optimunm=bt.optimize(n1=range(10,160,10),n2=range(10,160,10),constraint=lambda x:x.n2-x.n1>10, maximize='# Trades')
optimunm


Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

Start                     2018-01-01 00:00:00
End                       2024-01-25 00:00:00
Duration                   2215 days 00:00:00
Exposure Time [%]                   54.866667
Equity Final [$]                   4250.63775
Equity Peak [$]                   4298.387689
Return [%]                         325.063775
Buy & Hold Return [%]               91.271045
Return (Ann.) [%]                   27.520583
Volatility (Ann.) [%]               41.629717
Sharpe Ratio                          0.66108
Sortino Ratio                        1.538731
Calmar Ratio                         0.602916
Max. Drawdown [%]                  -45.645795
Avg. Drawdown [%]                   -7.448905
Max. Drawdown Duration      851 days 00:00:00
Avg. Drawdown Duration       83 days 00:00:00
# Trades                                   24
Win Rate [%]                        41.666667
Best Trade [%]                      74.889512
Worst Trade [%]                     -13.75332
Avg. Trade [%]                    

In [9]:
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1325', ...)

In [10]:
optimunm._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2,69,76,365.057994,334.799988,-60.516012,-0.082885,2018-04-13,2018-04-24,11 days
1,3,116,122,310.997994,272.000000,-116.993981,-0.125396,2018-06-20,2018-06-28,8 days
2,3,165,182,267.545988,242.300003,-75.737953,-0.094361,2018-08-30,2018-09-26,27 days
3,4,216,227,184.517994,177.949997,-26.271987,-0.035595,2018-11-16,2018-12-04,18 days
4,4,246,249,176.970000,169.000000,-31.880000,-0.045036,2019-01-01,2019-01-04,3 days
5,3,252,272,185.130000,176.000000,-27.390000,-0.049317,2019-01-09,2019-02-06,28 days
6,3,289,332,187.476003,186.199997,-3.828018,-0.006806,2019-03-05,2019-05-13,69 days
7,4,418,434,131.580000,118.050003,-54.119988,-0.102827,2019-09-17,2019-10-11,24 days
8,4,439,471,141.780000,170.350006,114.280024,0.201509,2019-10-18,2019-12-05,48 days
9,3,480,512,185.742006,163.500000,-66.726019,-0.119747,2019-12-18,2020-02-03,47 days


# **Task-3**

---



---



In [30]:
top_50=pd.read_csv("/content/drive/MyDrive/Documents/Top 50 in S&P 500(1).csv");
top_50.drop('Unnamed: 2',axis=1,inplace=True)
top_50.rename(columns={'Symbol (yfinance)': 'Ticker'},inplace=True)
top_50.head(50)

,Company,Ticker
0,Microsoft,MSFT
1,Apple,AAPL
2,Amazon,AMZN
3,Alphabet Class A,GOOGL
4,Tesla,TSLA
5,Berkshire Hathaway,BRK.B
6,Meta Platforms,META
7,NVIDIA,NVDA
8,Alphabet Class C,GOOG
9,UnitedHealth Group,UNH


In [71]:
import pandas_ta as ta
alpha_RSI=[0]*50
alpha_RSI=pd.DataFrame()
def get_stock_data(df,alpha_RSI):

  alpha_MACD=[0]*50
  for i in range(0,len(df)):
    data = yf.download(df.Ticker[i], start='2022-01-01', end='2023-01-01')
    alpha_RSI[i]=ta.rsi(data.Close, length=250)
    alpha_RSI.dropna(axis=0)


get_stock_data(top_50,alpha_RSI)
alpha_RSI=alpha_RSI.tail(1)
alpha_RSI

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******************

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-30,46.503382,46.008688,43.790917,44.637938,42.464208,None,42.391645,45.710374,44.659817,51.052492,...,50.689513,51.629546,47.946392,42.19673,53.212422,50.44146,47.908505,58.962998,50.136302,45.269782


In [78]:
alpha_RSI2=alpha_RSI.transpose()
alpha_RSI.dropna()
alpha_RSI2['Finally']=(alpha_RSI2['2022-12-30']-alpha_RSI2['2022-12-30'].min())/(alpha_RSI2['2022-12-30'].max()-alpha_RSI2['2022-12-30'].min())
alpha_RSI2.dropna()

Date,2022-12-30 00:00:00,Finally
0,46.503382,0.256864
1,46.008688,0.227359
2,43.790917,0.095083
3,44.637938,0.145602
4,42.464208,0.015953
6,42.391645,0.011625
7,45.710374,0.209566
8,44.659817,0.146907
9,51.052492,0.528189
10,55.385077,0.7866
